The code should be running with T4 gpu connection

In [1]:
%%capture
!pip install unsloth "xformers==0.0.28.post2"
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

# Install Flash Attention 2 for softcapping support
import torch
if torch.cuda.get_device_capability()[0] >= 8:
    !pip install --no-deps packaging ninja einops "flash-attn>=2.6.3"

In [2]:


from unsloth import FastLanguageModel ,is_bfloat16_supported
from datasets import load_dataset

import torch
from trl import SFTTrainer

from transformers import TrainingArguments
import time

from trl import  DataCollatorForCompletionOnlyLM

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


In [3]:

max_seq_length = 2048# Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "RanaHossny213/gamma_tuned-2b",
    max_seq_length # Add LoRA adapters so we only need to update 1 to 10% of all parameters!
= max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

==((====))==  Unsloth 2024.11.7: Fast Gemma patching. Transformers = 4.46.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.0+cu124. CUDA = 7.5. CUDA Toolkit = 12.4.
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.01G [00:00<?, ?B/s]

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


generation_config.json:   0%|          | 0.00/154 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/40.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/628M [00:00<?, ?B/s]

Unsloth 2024.11.7 patched 18 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


In [4]:
test_prompt = """Below is a description for a time series data. Write a response that gives the name of the best fitting machine learning algorithm in one word without explanation.
The best algorithm name should be one of this search space algorithms: AdaboostRegressor, ElasticNetRegressor,  ExtraTreesRegressor,  LassoRegressor,  LightgbmRegressor, SVR, GaussianProcessRegressor, RandomForestRegressor or  XGBoostRegressor.

### DESCRIPTION:
{}

### RESPONSE:"""


def formatting_test_prompts_func(examples):
    global tokenizer

    inputs = examples["series_description"]
    texts = []
    for input in  inputs:
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = test_prompt.format( input)
        texts.append(text)
    return { "text" : texts }

In [12]:
#please put the dic of test data , i try with train data just for test the running code
dataset = load_dataset('csv', data_files="/content/Regression_Univariate_train.csv")


In [15]:
test_dataset = dataset['train']
test_dataset = test_dataset.map(formatting_test_prompts_func, batched = True)
test_dataset

Dataset({
    features: ['dataset_name', 'series_description', 'algorithm', 'hyperparameters', 'text'],
    num_rows: 828
})

In [17]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[test_dataset['text'][0]], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

['<bos>Below is a description for a time series data. Write a response that gives the name of the best fitting machine learning algorithm in one word without explanation.\nThe best algorithm name should be one of this search space algorithms: AdaboostRegressor, ElasticNetRegressor,  ExtraTreesRegressor,  LassoRegressor,  LightgbmRegressor, SVR, GaussianProcessRegressor, RandomForestRegressor or  XGBoostRegressor.\n\n### DESCRIPTION:\nA univariate time-series dataset  consists of 648 samples with a missing values percentage of 0.0% imputed using FBProphet model and 0.0% detected outliers. The target series has a sampling rate of 480 minutes, minimum value of -1.281900892748634, maximum value of 1.2575716095378922, median value of -0.840998218756287, mean value of -0.3999772916658776, and average standard deviation of 0.22521469337610456 for the 10 percentiles. The series is detected as non-stationary using dickey fuller testand it turns into a stationary series using first order differe

In [18]:
inputs = tokenizer(
[test_dataset['text'][1]], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 5, use_cache = True)
tokenizer.batch_decode(outputs)

['<bos>Below is a description for a time series data. Write a response that gives the name of the best fitting machine learning algorithm in one word without explanation.\nThe best algorithm name should be one of this search space algorithms: AdaboostRegressor, ElasticNetRegressor,  ExtraTreesRegressor,  LassoRegressor,  LightgbmRegressor, SVR, GaussianProcessRegressor, RandomForestRegressor or  XGBoostRegressor.\n\n### DESCRIPTION:\nA univariate time-series dataset  consists of 5760 samples with a missing values percentage of 0.0% imputed using FBProphet model and 4.826388888888889% detected outliers. The target series has a sampling rate of 30 minutes, minimum value of -1.3327249393311884, maximum value of 1.3290006441545716, median value of -0.837102440111593, mean value of -0.5832298857283066, and average standard deviation of 0.19129701117888212 for the 10 percentiles. The series is detected as stationary using dickey fuller test.The series has 9 significant lags observed using th

In [19]:
test_responses=[]
# get all test data inference result
for test_prompt in test_dataset['text']:
  inputs= tokenizer(
  [test_prompt], return_tensors = "pt").to("cuda")

  outputs = model.generate(**inputs, max_new_tokens = 5, use_cache = True)
  test_responses.append(tokenizer.batch_decode(outputs))


In [20]:
# Convert the dataset to a Pandas DataFrame
df = test_dataset.to_pandas()
df['model_responses']= test_responses
df.to_csv('test_model_result_unsloth.csv', index=False)

In [21]:
predictions = []
for response in test_responses:
    result = response[0].split('\n\n### RESPONSE:')[1].split('</s>')[0].strip()
    result = result.replace('\n', '').replace('.', '')  # Remove \n and .
    if result == '<eos>LIGHTgbmRegressor':
        result = '<eos>LightgbmRegressor'
    elif result == '<eos>ADABoostRegressor':
        result = '<eos>AdaboostRegressor'

    predictions.append(result)

predictions


['XGBoostRegressor',
 'XGBoostRegressor',
 'XGBoostRegressor',
 'LassoRegressor',
 'LassoRegressor',
 'LassoRegressor',
 'ElasticNetRegressor',
 'ElasticNetRegressor',
 'ElasticNetRegressor',
 'ExtraTreesRegressor',
 'ElasticNetRegressor',
 'ElasticNetRegressor',
 'XGBoostRegressor',
 'LassoRegressor',
 'ElasticNetRegressor',
 'RandomForestRegressor',
 'LassoRegressor',
 'LassoRegressor',
 'ElasticNetRegressor',
 'XGBoostRegressor',
 'ExtraTreesRegressor',
 'ElasticNetRegressor',
 'XGBoostRegressor',
 'LassoRegressor',
 'RandomForestRegressor',
 'ElasticNetRegressor',
 'LassoRegressor',
 'ExtraTreesRegressor',
 'LassoRegressor',
 'RandomForestRegressor',
 'XGBoostRegressor',
 'LassoRegressor',
 'ElasticNetRegressor',
 'XGBoostRegressor',
 'LassoRegressor',
 'XGBoostRegressor',
 'XGBoostRegressor',
 'LassoRegressor',
 'LassoRegressor',
 'LassoRegressor',
 'LassoRegressor',
 'LassoRegressor',
 'LassoRegressor',
 'ExtraTreesRegressor',
 'XGBoostRegressor',
 'ElasticNetRegressor',
 'Elasti

In [24]:
len(predictions)

828

# run this only if the output of model contain the end tokenizer <eos>

In [ ]:
import re

def extract_model_name(prediction):
    """
    Extract the model name from a prediction string using regex.
    Corrects 'ADABoostRegressor' to 'AdaBoostRegressor'.
    """
    # Adjust regex to match model names more precisely
    match = re.search(r"\b[A-Za-z]+(?:[A-Za-z0-9]*)\b", prediction)
    if match:
        model_name = match.group()
        return model_name
    return None  # Return None if no match is found

# Apply the extraction to all predictions
extracted_predictions = [extract_model_name(pred.split('<eos>')[-1].strip()) for pred in predictions]

# Print the extracted model names
print(extracted_predictions)


# continue testing

In [22]:
actual_data= df['algorithm']
len(actual_data)

828

In [23]:
actual_data

,algorithm
0,XGBoostRegressor
1,ExtraTreesRegressor
2,RandomForestRegressor
3,LassoRegressor
4,RandomForestRegressor
...,...
823,XGBoostRegressor
824,LightgbmRegressor
825,AdaboostRegressor
826,ElasticNetRegressor


In [26]:
from sklearn.metrics import f1_score

# Assuming `predictions` and `actual_data` are lists or arrays of labels
f1 = f1_score(actual_data, predictions, average='micro')  # Use 'macro' or 'micro' as needed


print("F1 Score:", f1)


F1 Score: 0.2922705314009662


In [27]:
from sklearn.metrics import accuracy_score
# Calculate the accuracy
accuracy = accuracy_score(actual_data, predictions)

# Print the accuracy
print("Accuracy:", accuracy)

Accuracy: 0.2922705314009662
